In [1]:

import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.io import MemoryFile
from shapely.geometry import mapping


In [2]:

# Load CHM
with rasterio.open("chm_clipped_resampled.tif") as src:
    chm_data = np.squeeze(src.read(1))
    chm_profile = src.profile
    chm_crs = src.crs

# Load and clip NAIP to meadow extent
meadow_gdf = gpd.read_file("../meadow_extent.geojson").to_crs(chm_crs)
with rasterio.open("naip_clipped_resampled.tif") as src:
    naip_data, _ = mask(src, meadow_gdf.geometry, crop=True, nodata=0)
    naip_data = np.transpose(naip_data, (1, 2, 0)).astype(np.float32)
    naip_data[naip_data == 0] = np.nan


In [27]:

# Calculate indices
n, r, b = naip_data[:, :, 0], naip_data[:, :, 1], naip_data[:, :, 2]
brightness = n + r + b
ndvi = (n - r) / (n + r)
ndwi = (r - n) / (r + n)
avg = (n + r + b) / 3
bri_avg = (n - avg) + (r - avg) + (b - avg) / avg
water = (bri_avg < 0) & (ndwi > 0)
bare_mask = (brightness > 320) & (chm_data < 0.5) & (ndvi < 0.05) & ~water


In [29]:

# Classify bare vs non-bare
bare_classified = np.where(bare_mask, 1, 2).astype(np.uint8)
bare_classified = np.expand_dims(bare_classified, axis=0)


In [30]:


# Store bare_classified in memory
with MemoryFile() as memfile:
    with memfile.open(**chm_profile) as mem:
        mem.write(bare_classified)

        # Buffer polyline and mask inside buffer
        channel_gdf = gpd.read_file("../lacey_plong_simp.geojson").to_crs(chm_crs)
        buffered_geom = channel_gdf.geometry.buffer(10)
        out_image, _ = mask(
            mem,
            [mapping(geom) for geom in buffered_geom],
            invert=True,  # Keep data outside buffer
            crop=False,
            filled=True,
            nodata=0
        )

        # Calculate bare earth area
        pixel_area = mem.transform.a ** 2
        bare_earth_area = np.sum(out_image == 1) * pixel_area



In [36]:

# Load CHM again for vegetation stats
with rasterio.open("CHM_clipped_resampled.tif") as src:
    chm = src.read(1)
    pixel_area = src.transform.a ** 2
    trees = np.sum(chm > 3) * pixel_area
    shrubs = np.sum((chm > 0.2) & (chm < 3)) * pixel_area
    total_area = np.sum(~np.isnan(chm)) * pixel_area


In [37]:

# Print summary
print("Bare Earth Cover (%):", round(bare_earth_area / total_area * 100, 2))
print("Tree Cover (%):", round(trees / total_area * 100, 2))
print("Shrub Cover (%):", round(shrubs / total_area * 100, 2))


Bare Earth Cover (%): 1.68
Tree Cover (%): 2.68
Shrub Cover (%): 9.06


## need a way to discern conifer from deciduous

In [1]:

def sum_chm_by_internal_buffers_with_export(chm_path, boundary_path, buffer_step=10,
                                    height_threshold=3, output_geojson="internal_buffers.geojson"):
    
    # Load boundary and CHM
    gdf = gpd.read_file(boundary_path)
    with rasterio.open(chm_path) as src:
        chm = src.read(1)
        #chm[chm <= 0] = np.nan
        chm_crs = src.crs

        if gdf.crs != chm_crs:
            gdf = gdf.to_crs(chm_crs)

        geom = gdf.geometry.iloc[0]

        results = []
        buffer_geoms = []
        distance = 0

        while True:
            buffered = geom.buffer(-distance)
            if buffered.is_empty or not buffered.is_valid:
            #if buffered.is_empty or not buffered.is_valid or not isinstance(buffered, Polygon):
                break

            try:
                out_image, _ = mask(src, [mapping(buffered)], crop=True, nodata=np.nan)
                out_image = np.squeeze(out_image)
                total_area = np.sum(~np.isnan(out_image))
                trees = np.nansum(out_image > height_threshold)
                #trees = np.where(out_image > height_threshold, out_image, 0)
                #total = np.sum(trees > 0)
                results.append((distance, trees, total_area))
                buffer_geoms.append(buffered)
            except ValueError:
                break

            distance += buffer_step

    # Save buffers as GeoJSON
    buffer_gdf = gpd.GeoDataFrame({'distance_m': [r[0] for r in results], 'sum_gt_3m': [r[1] for r in results], 'total_area': [r[2] for r in results]}, geometry=buffer_geoms, crs=chm_crs)
    buffer_gdf.to_file(output_geojson, driver="GeoJSON")

    return results, output_geojson


In [4]:

results, geojson_path = sum_chm_by_internal_buffers_with_export(
    "CHM_clipped_resampled.tif",
    "../meadow_extent.geojson",
    buffer_step=10,
    height_threshold=5,
    output_geojson="internal_buffers.geojson"
)

print(f"Saved {len(results)} internal buffers to {geojson_path}")


Saved 40 internal buffers to internal_buffers.geojson


In [5]:
import pandas as pd
results_df = pd.DataFrame({'distance_m': [r[0] for r in results], 'sum_gt_3m': [r[1] for r in results], 'total_area': [r[2] for r in results]})

In [6]:
results_df

,distance_m,sum_gt_3m,total_area
0,0,101473,4803709
1,10,35744,4508259
2,20,21362,4212043
3,30,15502,3933405
4,40,11665,3676966
5,50,9168,3434665
6,60,6922,3210725
7,70,5129,3005686
8,80,3675,2820775
9,90,2678,2643806


In [7]:
results_df['buffer_chm_sum'] = results_df.sum_gt_3m - results_df.sum_gt_3m.shift(-1)

In [8]:
results_df['buffer_total_sum'] = results_df.total_area - results_df.total_area.shift(-1)

In [9]:
results_df['percent_cover_3m'] = round(results_df.buffer_chm_sum / results_df.buffer_total_sum * 100,2)

In [10]:
results_df

,distance_m,sum_gt_3m,total_area,buffer_chm_sum,buffer_total_sum,percent_cover_3m
0,0,101473,4803709,65729.0,295450.0,22.25
1,10,35744,4508259,14382.0,296216.0,4.86
2,20,21362,4212043,5860.0,278638.0,2.10
3,30,15502,3933405,3837.0,256439.0,1.50
4,40,11665,3676966,2497.0,242301.0,1.03
5,50,9168,3434665,2246.0,223940.0,1.00
6,60,6922,3210725,1793.0,205039.0,0.87
7,70,5129,3005686,1454.0,184911.0,0.79
8,80,3675,2820775,997.0,176969.0,0.56
9,90,2678,2643806,540.0,172308.0,0.31
